In [ ]:
from inait_forecasting_client import (
    predict,
    backtest,
    plot,
    compare,
    compare_plot,
)
from utils import read_aapl_vola

### Import dataset and define config 

In [ ]:
data = read_aapl_vola()

start_date = "2016-03-01"
end_date = "2016-10-31"

config_common = {
    "data": data,
    "target_columns": "AAPL",
    "explanatory_columns": [
        "AAPL__hl",
        "AAPL__co",
        "AAPL__ocp",
        "AAPL__log_returns",
        #'AAPL__lret_squared',
        "AAPL__earning_horizon1",
        "AAPL__earning_horizon2",
        "AAPL__earning_horizon3",
        "AAPL__earning_horizon4",
    ],
    "forecasting_horizon": 4,
    "target_transformer": "NO_TARGET_TRANSFORMER",
    "model": "inait-best",
}
config_predict = config_common.copy()
config_predict.update(
    {
        "session_id": "demo_vola_best_predict_v0",
        "historical_data_end_date": end_date,
    }
)
config_backtest = config_common.copy()
config_backtest.update(
    {
        "data": data,
        "start_date": start_date,
        "end_date": end_date,
        "session_id": "demo_vola_best_dates_v0",  # cached session id
    }
)

### 1. Predict

In [ ]:
predict_results = predict(**config_predict)
plot(
    historical_data=data.loc[:end_date, config_predict["target_columns"]],
    predicted_data=predict_results["prediction"],
)

### 2. Backtest

In [ ]:
backtest_results = backtest(**config_backtest)
plot(
    historical_data=data.loc[:, config_backtest["target_columns"]],
    predicted_data=backtest_results["prediction"],
)

### 3. Compare

In [ ]:
session_ids = {
    "inait-best": backtest_results["session_id"],
    # "ridge": None,
    "fast_boost": "demo_vola_fast_boost_dates_v0",
    "naive": "demo_vola_naive_dates_v0",  # "demo_mean1_v0",
    "mean_5": "demo_vola_mean5_dates_v0",  # "demo_mean5_v0",
    "mean_10": "demo_vola_mean10_dates_v",  # "demo_mean10_v0"
}  # cached session ids for demo

In [ ]:
compare_results = compare(
    models=session_ids.keys(), session_ids=session_ids, **config_backtest
)
compare_plot(compare_results["scores_df"], plot_type="bar")

### Extra: Predict using optimal model based on backtest result

In [ ]:
if compare_results["optimal_model"] == "inait-best":
    print("The optimal model is the same as the backtested model.")
    config_optimal = config_predict.copy()
else:
    print(f"Running predictions with optimal model: {compare_results['optimal_model']}")
    config_optimal = config_common.copy()
    config_optimal["model"] = compare_results["optimal_model"]

if compare_results["optimal_model"] == "naive" or compare_results[
    "optimal_model"
].startswith("mean"):
    raise ValueError("Naive and Mean baselines are not yet supported for predict.")
else:
    optimal_predict_results = predict(**config_optimal)
    plot(
        historical_data=data.loc[:end_date, config_optimal["target_columns"]],
        predicted_data=optimal_predict_results["prediction"],
    )